In [1]:
import os
from scipy.linalg import svd 
from IPython.core.display import SVG
import pyomo.environ as pyo
import idaes
import idaes.logger as idaeslog
from idaes.core.util.model_statistics import degrees_of_freedom as dof
import idaes.core.util.exceptions as idaes_except
from idaes.core.solvers import use_idaes_solver_configuration_defaults
import idaes.core.util.scaling as iscale
import soec_standalone_flowsheet
from soec_flowsheet_costing import (
    get_solo_soec_capital_costing,
    initialize_flowsheet_costing,
    scale_flowsheet_costing,
    get_soec_OM_costing,
    display_soec_costing
)
import idaes.core.util.model_statistics as mstat
from idaes.models.properties import iapws95
import idaes.core.util.unit_costing as costing
from pyomo.util.check_units import assert_units_consistent
import idaes.core.util as iutil

import numpy as np
import pandas as pd

ModuleNotFoundError: No module named 'idaes.core.util.unit_costing'

In [ ]:
from pyomo.core.expr.current import identify_variables
from pyomo.common.collections import ComponentSet
def find_active_constraints_containing_variable(var, blk):
    con_set = ComponentSet()
    CUID = pyo.ComponentUID(var)
    for c in blk.component_data_objects(ctype=pyo.Constraint, active=True, descend_into=True):
        for v in identify_variables(c.body):
            if CUID.matches(v):
                con_set.add(c)
    return con_set

In [ ]:
use_idaes_solver_configuration_defaults()
idaes.cfg.ipopt.options.nlp_scaling_method = "user-scaling"
idaes.cfg.ipopt.options.OF_ma57_automatic_scaling = "yes"
idaes.cfg.ipopt["options"]["linear_solver"] = "ma57"
idaes.cfg.ipopt["options"]["max_iter"] = 300
idaes.cfg.ipopt["options"]["halt_on_ampl_error"] = "no"

In [ ]:
m = pyo.ConcreteModel()
m.fs = soec_standalone_flowsheet.SoecStandaloneFlowsheet(default={"dynamic":False})
iscale.calculate_scaling_factors(m)

In [ ]:
m.fs.initialize_build(outlvl=idaeslog.INFO_LOW)#, load_from="soec_standalone_init.json.gz")

In [ ]:
print(dof(m))
get_solo_soec_capital_costing(m.fs)
get_soec_OM_costing(m.fs)
print(dof(m))
iscale.calculate_scaling_factors(m)

In [ ]:
solver = pyo.SolverFactory("ipopt")

In [ ]:
initialize_flowsheet_costing(m.fs)
scale_flowsheet_costing(m.fs)

m.fs.tags_output["annual_water_cost"] = iutil.ModelTag(
    doc="Annual water cost",
    expr=m.fs.costing.annual_water_cost,
    format_string="{:.2f}",
)

In [ ]:
solver.solve(m, tee=True, options={"tol": 1e-6, "max_iter": 300, "halt_on_ampl_error":"no"})

In [ ]:
def set_indexed_variable_bounds(var,bounds):
    for idx, subvar in var.items():
        subvar.bounds = bounds
if True:
    m.fs.obj = pyo.Objective(
        expr = (
            m.fs.costing.annual_electricity_cost 
            + m.fs.costing.annual_water_cost
            + m.fs.costing.total_annualized_cost
            + m.fs.costing.annual_fixed_operations_and_maintenance_cost
            + m.fs.costing.annual_air_cost
        )
    )
    
    for hx in [m.fs.feed_hot_exchanger, m.fs.sweep_hot_exchanger, m.fs.sweep_medium_exchanger,
              m.fs.water_evaporator01, m.fs.water_evaporator02, m.fs.water_evaporator03,
              m.fs.water_evaporator04, m.fs.water_evaporator05, m.fs.water_preheater]:
        set_indexed_variable_bounds(hx.delta_temperature_in, (0, None))
        set_indexed_variable_bounds(hx.delta_temperature_out, (0, None))
        hx.area.unfix()
    
    for hx in [m.fs.water_evaporator01, m.fs.water_evaporator02]:
            try:
                tdew = hx.shell.properties_out[0].temperature_dew["Liq", "Vap"]
            except KeyError:
                tdew = hx.shell.properties_out[0].temperature_dew["Vap", "Liq"]

            @hx.shell.Constraint(m.fs.time)
            def no_condensation_eqn(b, t):
                return b.properties_out[0].temperature >= tdew
            iscale.constraint_scaling_transform(hx.shell.no_condensation_eqn[0], 1e-2)
            
    for hx in [m.fs.water_evaporator03, m.fs.water_evaporator04, m.fs.water_preheater]:
        for t in m.fs.time:
            try:
                eps = hx.hot_side.properties_in[t].eps_2_Liq_Vap
            except (KeyError, idaes_except.PropertyNotSupportedError):
                eps = hx.hot_side.properties_in[t].eps_2_Vap_Liq
            eps = 1e-3
    
    m.fs.h2_mass_production.fix(2)
    m.fs.water_preheater.tube_inlet.flow_mol.unfix()
    m.fs.soec_single_pass_water_conversion.unfix()
    m.fs.feed_recycle_split.split_fraction.unfix()
    m.fs.sweep_recycle_split.split_fraction.unfix()
    m.fs.sweep_blower.inlet.flow_mol.unfix()
    #m.fs.sweep_blower.control_volume.properties_out[:].pressure.unfix()
    m.fs.feed_heater.outlet.temperature.unfix()
    m.fs.sweep_heater.outlet.temperature.unfix()
    
    
    
    m.fs.soec_module.potential_cell.unfix()
    m.fs.soec_module.number_cells.unfix()
    m.fs.costing.electricity_price.fix(71.7)
    
    set_indexed_variable_bounds(m.fs.soec_module.solid_oxide_cell.potential, (1.1,1.3))
    set_indexed_variable_bounds(m.fs.water_split.split_fraction, (0.03,0.98))
    set_indexed_variable_bounds(m.fs.feed_heater.heat_duty,(0,None))
    set_indexed_variable_bounds(m.fs.sweep_heater.heat_duty,(0,None))
    set_indexed_variable_bounds(m.fs.heat_pump_hot_terminus.heat_duty, (0, None))
    set_indexed_variable_bounds(m.fs.sweep_blower.inlet.flow_mol,(1000,None))
    m.fs.feed_recycle_split.split_fraction[0,"recycle"].bounds = (0.03,0.5)
    m.fs.sweep_recycle_split.split_fraction[0,"recycle"].bounds = (0.03,0.5)
    set_indexed_variable_bounds(m.fs.soec_overall_water_conversion, (0.4,0.8))
    set_indexed_variable_bounds(m.fs.soec_module.solid_oxide_cell.temperature_z, (625+273.15,750+273.15))
    set_indexed_variable_bounds(m.fs.soec_module.solid_oxide_cell.fuel_channel.temperature_inlet, (600+273.15,750+273.15))
    set_indexed_variable_bounds(m.fs.soec_module.solid_oxide_cell.oxygen_channel.temperature_inlet, (600+273.15,750+273.15))
    #set_indexed_variable_bounds(m.fs.soec_module.solid_oxide_cell.current_density, (-8000,8000))
    set_indexed_variable_bounds(m.fs.heat_source.inlet.flow_mol[0], (1, None))
    
    for cmp in [m.fs.cmp01, m.fs.cmp02, m.fs.cmp03, m.fs.cmp04]:
        cmp.ratioP.unfix()
        set_indexed_variable_bounds(cmp.ratioP, (1, 3))
    
    @m.fs.Constraint(m.fs.time)
    def thermal_gradient_eqn_1(b, t):
        return (b.soec_module.fuel_outlet.temperature[t] - b.soec_module.fuel_inlet.temperature[t]) <= 50

    @m.fs.Constraint(m.fs.time)
    def thermal_gradient_eqn_2(b, t):
        return (b.soec_module.fuel_outlet.temperature[t] - b.soec_module.fuel_inlet.temperature[t]) >= -50

    @m.fs.Constraint(m.fs.time)
    def thermal_gradient_eqn_3(b, t):
        return (b.soec_module.oxygen_outlet.temperature[t] - b.soec_module.oxygen_inlet.temperature[t]) <= 50

    @m.fs.Constraint(m.fs.time)
    def thermal_gradient_eqn_4(b, t):
        return (b.soec_module.oxygen_outlet.temperature[t] - b.soec_module.oxygen_inlet.temperature[t]) >= -50

    @m.fs.Constraint(m.fs.time)
    def thermal_gradient_eqn_5(b, t):
        return (b.soec_module.fuel_outlet.temperature[t] - b.soec_module.oxygen_inlet.temperature[t]) <= 50

    @m.fs.Constraint(m.fs.time)
    def thermal_gradient_eqn_6(b, t):
        return (b.soec_module.fuel_outlet.temperature[t] - b.soec_module.oxygen_inlet.temperature[t]) >= -50

    @m.fs.Constraint(m.fs.time)
    def thermal_gradient_eqn_7(b, t):
        return (b.soec_module.oxygen_outlet.temperature[t] - b.soec_module.fuel_inlet.temperature[t]) <= 50

    @m.fs.Constraint(m.fs.time)
    def thermal_gradient_eqn_8(b, t):
        return (b.soec_module.oxygen_outlet.temperature[t] - b.soec_module.fuel_inlet.temperature[t]) >= -50

    iscale.constraint_scaling_transform(m.fs.thermal_gradient_eqn_1[0],1e-2)
    iscale.constraint_scaling_transform(m.fs.thermal_gradient_eqn_2[0],1e-2)
    iscale.constraint_scaling_transform(m.fs.thermal_gradient_eqn_3[0],1e-2)
    iscale.constraint_scaling_transform(m.fs.thermal_gradient_eqn_4[0],1e-2)
    iscale.constraint_scaling_transform(m.fs.thermal_gradient_eqn_5[0],1e-2)
    iscale.constraint_scaling_transform(m.fs.thermal_gradient_eqn_6[0],1e-2)
    iscale.constraint_scaling_transform(m.fs.thermal_gradient_eqn_7[0],1e-2)
    iscale.constraint_scaling_transform(m.fs.thermal_gradient_eqn_8[0],1e-2)
    
    @m.fs.Constraint(m.fs.time)
    def average_current_density_constraint(b, t):
        return m.fs.soec_module.solid_oxide_cell.average_current_density[0]/1000 >= -8
    
    @m.fs.Constraint(m.fs.time)
    def sweep_min_concentration_eqn(b,t):
        return b.sweep_recycle_split.inlet.mole_frac_comp[t,"O2"] >= 0.25 
    @m.fs.Constraint(m.fs.time)
    def sweep_max_concentration_eqn(b,t):
        return b.sweep_recycle_split.inlet.mole_frac_comp[t,"O2"] <= 0.35 
    
    @m.fs.Constraint(m.fs.time)
    def min_h2_feed_eqn(b,t):
        return b.feed_recycle_mix.outlet.mole_frac_comp[t,"H2"] >= 0.05
    
    @m.fs.Constraint(m.fs.time)
    def water_utilization(b, t):
        return (b.water_demand[0] - b.water_recycle[0]) <= 10475 * pyo.units.mol/pyo.units.s
#     @m.fs.Constraint(m.fs.time)
#     def equal_pressures_eqn(b,t):
#        return b.soec_module.fuel_inlet.pressure[t] == b.soec_module.oxygen_inlet.pressure[t]
    
#     iscale.constraint_scaling_transform(m.fs.equal_pressures_eqn[0],1e-5)

In [ ]:
#set_indexed_variable_bounds(m.fs.soec_module.solid_oxide_cell.current_density, (-8000,8000))
#m.fs.average_current_density_constraint.deactivate()
m.fs.h2_mass_production.fix(5)
solver.solve(m, tee=True, options={"tol": 3e-8, "max_iter": 300, "halt_on_ampl_error":"no"})

In [ ]:
m.fs.soec_module.number_cells.pprint()
m.fs.soec_module.solid_oxide_cell.current_density.pprint()
pyo.value((m.fs.obj)*1e6/(5*m.fs.costing.plant_uptime))

In [ ]:
display_soec_costing(m.fs)

In [ ]:
display(SVG(m.fs.write_pfd()))

In [ ]:
streams_df = m.fs.streams_dataframe()
display(streams_df)

In [ ]:
m.fs.write_pfd(fname="soec_standalone_design_point.svg")
streams_df.to_csv("soec_standalone_design_point.csv")

In [ ]:
water_heaters = [
    m.fs.water_evaporator01,
    m.fs.water_evaporator02,
    m.fs.water_evaporator03,
    m.fs.water_evaporator04,
    m.fs.water_evaporator05,
    m.fs.water_preheater,
]
cross_flow_exchangers = [
    m.fs.feed_hot_exchanger,
    m.fs.sweep_hot_exchanger,
    m.fs.sweep_medium_exchanger,
]
heaters = [
    m.fs.feed_heater,
    m.fs.sweep_heater
]
flash_vessels = [m.fs.product_flash01,
                 #m.fs.product_flash02,
                 m.fs.product_flash03, m.fs.product_flash04]



compressors = [getattr(m.fs, f"cmp0{i}") for i in range(1,5)]
print(f"Total CapEx: ${pyo.value(m.fs.costing.total_TPC):.2f}MM")
print(f"Contribution from SOEC: ${pyo.value(m.fs.soec_module.ersatz_costing.total_plant_cost):.2f}MM")
print(f"Contribution from H2 compressors: ${pyo.value(sum([cmp.ersatz_costing.total_plant_cost for cmp in compressors])):.2f}MM")
print(f"Contribution from H2O compressor: ${pyo.value(m.fs.water_compressor.costing.total_plant_cost):.2f}MM")
print(f"Contribution from trim heaters: ${pyo.value(sum([heater.ersatz_costing.total_plant_cost for heater in heaters])):.2f}MM")
print(f"Contribution from gas-gas exchangers: ${pyo.value(sum([hx.ersatz_costing.total_plant_cost for hx in cross_flow_exchangers])):.2f}MM")
print(f"Contribution from gas-water exchangers: ${pyo.value(sum([hx.costing.total_plant_cost for hx in water_heaters])):.2f}MM")
print(f"Contribution from flash vessels: ${pyo.value(sum([flash.costing.total_plant_cost for flash in flash_vessels])):.2f}MM")
print(f"Contribution from heat pump: ${pyo.value(m.fs.heat_pump.ersatz_costing.total_plant_cost):.2f}MM")
print(f"Contribution from sweep blower: ${pyo.value(m.fs.sweep_blower.costing.total_plant_cost):.2f}MM")
print(f"Contribution from water feed & treatment systems: ${pyo.value(m.fs.costing.water_systems_cost):.2f}MM")
print(f"Contribution from accessory electric equipment: ${pyo.value(m.fs.costing.accessory_electric_plant_cost):.2f}MM")
print(f"Contribution from instrumentation and controls: ${pyo.value(m.fs.costing.instrumentation_and_control_cost):.2f}MM")
print(f"Contribution from improvements to site: ${pyo.value(m.fs.costing.improvements_to_site_cost):.2f}MM")
print(f"Contribution from buildings and structures: ${pyo.value(m.fs.costing.buildings_and_structures_cost):.2f}MM")

print("\n")
print("------------------------------------------")
print("Fixed Costs for Optimized Design")
print("------------------------------------------")

# tpc = pyo.value(m.fs.costing.total_TPC)
# tasc = tpc*1.21*1.093
# ac = tasc*0.0707
print(f"TPC = {pyo.value(m.fs.costing.total_TPC)}")
print(f"TASC = {pyo.value(m.fs.costing.total_as_spent_cost)}")
print(f"Annualized TASC (MM$/yr) = {pyo.value(m.fs.costing.total_annualized_cost)}")

# Parameters
# n_op = 6.3
# hourly_rate = 38.50
# labor_burden = 30

# Fixed O&M components
# annual_op_labor = n_op * hourly_rate * 8760 * (1 - labor_burden/100)/1e6
# maint_labor = tpc * 0.4 * 0.019 
# maint_material = tpc * 0.6 * 0.019
# admin_labor = 0.25*(annual_op_labor + maint_labor)
# prop_tax_ins = 0.02*tpc
# soec_replace = pyo.value(4.2765*m.fs.soec_module.number_cells)/1e6

print("Fixed O&M Costs")
print(f"annual_op_labor (MM$/yr) = {pyo.value(m.fs.costing.annual_operating_labor_cost)}")
print(f"maint_labor (MM$/yr) = {pyo.value(m.fs.costing.maintenance_labor_cost)}")
print(f"maint_material (MM$/yr) = {pyo.value(m.fs.costing.maintenance_material_cost)}")
print(f"admin_labor (MM$/yr) = {pyo.value(m.fs.costing.admin_and_support_labor_cost)}")
print(f"prop_tax_ins (MM$/yr) = {pyo.value(m.fs.costing.property_tax_and_insurance_cost)}")
print(f"soec_replace (MM$/yr) = {pyo.value(m.fs.soec_module.ersatz_costing.annual_soec_replacement_cost)}")
# total_fixed = annual_op_labor + maint_labor + maint_material + admin_labor + prop_tax_ins + soec_replace
print(f"Annualized Fixed O&M (MM$/year) = {pyo.value(m.fs.costing.annual_fixed_operations_and_maintenance_cost)}")
fixed_cost_per_h2 = pyo.value(
    m.fs.costing.total_annualized_cost
    + m.fs.costing.annual_fixed_operations_and_maintenance_cost
)/24/365.2425/5/60/60*1e6
print(f"Fixed ($/kg h2) = {fixed_cost_per_h2}")

print(f"Annual water cost MM$ {pyo.value(m.fs.costing.annual_water_cost)}")
print(f"water cost $/kg h2 {pyo.value(m.fs.costing.annual_water_cost)/24/365/5/60/60*1e6}")

In [ ]:
for hx in water_heaters + cross_flow_exchangers:
    print(hx)
    hx.area.display()
    hx.area.fix()
m.fs.soec_module.number_cells.fix()
m.fs.soec_module.number_cells.display()

print(m.fs.feed_recycle_split)
m.fs.feed_recycle_split.split_fraction.display()
print(m.fs.sweep_recycle_split)
m.fs.sweep_recycle_split.split_fraction.display()
print(m.fs.sweep_blower)
m.fs.sweep_blower.inlet.flow_mol.display()
m.fs.feed_heater.outlet.temperature.display()
m.fs.sweep_heater.outlet.temperature.display()
m.fs.soec_module.potential_cell.display()

In [ ]:
for flash in flash_vessels:
    flash.diameter.fix()
    flash.length.fix()
    flash.length_diameter_heuristic.deactivate()
    flash.capacity_heuristic.deactivate()
    
m.fs.obj.deactivate()
m.fs.obj2 = pyo.Objective(
        expr = (
            m.fs.costing.annual_electricity_cost 
            + m.fs.costing.annual_water_cost
            + m.fs.costing.annual_air_cost
        )
    )    

In [ ]:
run_samples = True
df = None
prod_vec = np.linspace(5, 1, 33)

if run_samples:
    i=1
    df = pd.DataFrame(columns=m.fs.tags_output.table_heading())
    for prod in prod_vec:
        print(prod)
        m.fs.h2_mass_production.fix(prod)
        res = solver.solve(m, tee=True, options={"tol": 1e-6, "max_iter": 500, "halt_on_ampl_error":"no"})
        assert res.solver.termination_condition == pyo.TerminationCondition.optimal
        assert res.solver.status == pyo.SolverStatus.ok
        df.loc[i] = m.fs.tags_output.table_row(numeric=True)
        i += 1

In [ ]:
if df is not None:
    df.to_csv("soec.csv")

In [ ]:
display(SVG(m.fs.write_pfd()))

In [ ]:
for flash in [m.fs.product_flash01, 
              #m.fs.product_flash02,
              m.fs.product_flash03, m.fs.product_flash04]:
    print(flash.name)
    print(f"Diameter: {flash.diameter.value}m")
    print(f"Height: {flash.length.value}m")
    print()

In [ ]:
m.fs.soec_module.solid_oxide_cell.temperature_z.pprint()

In [ ]:
print(df)

In [ ]:
pyo.value(m.fs.product_flash01.liq_outlet.flow_mol[0])